## Introduction and Schema Diagram

In [1]:
%%capture
%load_ext sql
%sql sqlite:///chinook.db

'Connected: None@chinook.db'

## Overview of the Data

In [2]:
%%sql
SELECT
    name,
    type
FROM sqlite_master
WHERE type IN ("table","view");

Done.


name,type
album,table
artist,table
customer,table
employee,table
genre,table
invoice,table
invoice_line,table
media_type,table
playlist,table
playlist_track,table


In [3]:
%%sql 
SELECT * FROM album
LIMIT 10;

Done.


album_id,title,artist_id
1,For Those About To Rock We Salute You,1
2,Balls to the Wall,2
3,Restless and Wild,2
4,Let There Be Rock,1
5,Big Ones,3
6,Jagged Little Pill,4
7,Facelift,5
8,Warner 25 Anos,6
9,Plays Metallica By Four Cellos,7
10,Audioslave,8


In [4]:
%%sql
with USA_Tracks as (
SELECT il.track_id,count(il.track_id) tracks_count FROM customer c 
INNER JOIN invoice i on i.customer_id = c.customer_id 
INNER JOIN invoice_line il on  il.invoice_id = i.invoice_id
WHERE c.country = 'USA'
group by il.track_id),

track_genre as (
select t.track_id, g.name genre from track t 
inner join genre g on t.genre_id = g.genre_id 
)

select tg.genre, sum(tracks_count) tracks_sold, 
round(cast(sum(tracks_count) as float)*100/(select sum(tracks_count) from usa_tracks),3) percentage_sold
from usa_tracks ut
inner join track_genre tg on tg.track_id = ut.track_id
group by tg.genre
order by 2 desc



Done.


genre,tracks_sold,percentage_sold
Rock,561,53.378
Alternative & Punk,130,12.369
Metal,124,11.798
R&B/Soul,53,5.043
Blues,36,3.425
Alternative,35,3.33
Latin,22,2.093
Pop,22,2.093
Hip Hop/Rap,20,1.903
Jazz,14,1.332




Based on the sales of tracks across different genres in the USA, we should purchase the new albums by the following artists:

   - Red Tone (Punk)
   - Slim Jim Bites (Blues)
   - Meteor and the Girls (Pop)

It's worth keeping in mind that combined, these three genres only make up only 17% of total sales, so we should be on the lookout for artists and albums from the 'rock' genre, which accounts for 53% of sales.


## Analyzing Employee Sales Performance

In [5]:
%%sql 
select e.first_name||' '||e.last_name employee, e.hire_date, round(sum(i.total),3) total_sales from customer c 
inner join (select  customer_id, sum(total) total from invoice group by customer_id )as i on i.customer_id = c.customer_id
inner join employee e on e.employee_id = c.support_rep_id 
group by 1 ,2
order by 3 desc

Done.


employee,hire_date,total_sales
Jane Peacock,2017-04-01 00:00:00,1731.51
Margaret Park,2017-05-03 00:00:00,1584.0
Steve Johnson,2017-10-17 00:00:00,1393.92


While there is a 20% difference in sales between Jane (the top employee) and Steve (the bottom employee), the difference roughly corresponds with the differences in their hiring dates.

%%sql
select distinct(country) from customer

## Purchases by country

In [43]:
%%sql 
with Cntry_total as (select c.country,count(distinct(c.customer_id)) cnt ,sum(total) total
from Invoice i 
left  join Customer c on i.customer_id = c.customer_id 
group by c.country),

country_count as (select s.*, 
case 
    when s.cnt =1 then 'other'
    else s.country
    end as cntry

from Cntry_total s),

Country_Order as (select cntry, sum(cnt) CNT ,sum(total) TOTAL,
case 
when cntry = 'other' then 1 
else 0
end  as sort 
from country_count
group by cntry)


SELECT CO.CNTRY 'Country', Co.cnt " Customers", Co.Total "total_sales", co.total/co.cnt customer_lifetime_value
 FROM COUNTRY_ORDER CO
ORDER BY CO.SORT ,CO.TOTAL desc

Done.


Country,Customers,total_sales,customer_lifetime_value
USA,13,1040.4899999999998,80.0376923076923
Canada,8,535.5900000000001,66.94875000000002
Brazil,5,427.68000000000006,85.53600000000002
France,5,389.0699999999999,77.81399999999998
Germany,4,334.62,83.655
Czech Republic,2,273.24000000000007,136.62000000000003
United Kingdom,3,245.52,81.84
Portugal,2,185.13000000000002,92.56500000000001
India,2,183.14999999999998,91.57499999999999
other,15,1094.9399999999998,72.996




Based on the data, there may be opportunity in the following countries:

  - Czech Republic
  - United Kingdom
  - India

It's worth keeping in mind that because the amount of data from each of these countries is relatively low. Because of this, we should be cautious spending too much money on new marketing campaigns, as the sample size is not large enough to give us high confidence. A better approach would be to run small campaigns in these countries, collecting and analyzing the new customers to make sure that these trends hold with new customers.


## Albums vs Individual Tracks

In [150]:
%%sql 
with invoice_album as 
(select  il.invoice_id,t.album_id,t.track_id from invoice_line il
inner join track t on t.track_id = il.track_id ),

invoice_tracks as (select ia.invoice_id, t.track_id  from track t  
inner join invoice_album ia on  t.album_id  = ia.album_id  ),

album_tracks as (select track_id,i.invoice_id from invoice i inner join invoice_line
              il on il.invoice_id = i.invoice_id 
             ),

album_purchase as (select i.invoice_id , 
case  when 

(select track_id  from invoice_tracks t1 where t1.invoice_id = i.invoice_id 
 except
select track_id  from album_tracks t2 where t2.invoice_id = i.invoice_id 
 ) is null

and 
(select track_id  from album_tracks t2 where t2.invoice_id = i.invoice_id 
 except
select track_id  from invoice_tracks t1 where t1.invoice_id = i.invoice_id 
 ) is null



 then 'Yes'
else 'No' 
end as album_purchase
from invoice i)


select album_purchase,count(album_purchase) number_of_invoices,
    cast(count(album_purchase) as float)/(select count(*) 
    from album_purchase) percent
from album_purchase a
group by 1



Done.


album_purchase,number_of_invoices,percent
No,500,0.8143322475570033
Yes,114,0.18566775244299674




Album purchases account for 18.6% of purchases. Based on this data, I would recommend against purchasing only select tracks from albums from record companies, since there is potential to lose one fifth of revenue.


## Artists and Playlists 

In [163]:
%%sql 

with playlist_artist as (
select  ar.name, pl.playlist_id   from playlist pl
inner join playlist_track plt on plt.playlist_id = pl.playlist_id 
inner join track t on t.track_id = plt.track_id 
inner join album al on al.album_id = t.album_id 
inner join artist ar on ar.artist_id = al.artist_id )


select pla.name Artist,count(*)playlist_count,cast(count(*) as float)*100 /(select count(*) from playlist_artist) percent
from playlist_artist pla
group by 1 
order by 2 desc
limit 10




Done.


Artist,playlist_count,percent
Iron Maiden,516,5.920826161790017
U2,333,3.820998278829604
Metallica,296,3.3964429145152035
Led Zeppelin,252,2.891566265060241
Deep Purple,226,2.5932300631095813
Lost,184,2.1113023522662075
Pearl Jam,177,2.0309810671256456
Eric Clapton,145,1.663798049340218
Faith No More,145,1.663798049340218
Lenny Kravitz,143,1.6408491107286287


Iron Maiden's tracks are more popular in playlists

## Purchased Vs not Purchased 

In [193]:
%%sql
with invoice_tracks as
(select distinct(track_id) from invoice_line),



Purchases as (select 
case 
 when (select it.track_id from invoice_tracks it where t.track_id =it.track_id  ) is  null then "No"
else 'Yes'    
end as 'Purchased'
from album a 
left  join track t on a.album_id = t.album_id )

select p.purchased, count(*) 'Track Count', cast(count(*) as float)/(select count(*) from Purchases)  Percent
from Purchases p 
group by p.purchased



Done.


Purchased,Track Count,Percent
No,1697,0.4844419069369112
Yes,1806,0.5155580930630888


 Almost half of the tracks were never purchased

## Protected Vs Not Protected

In [207]:
%%sql
with track_media as 
(select t.track_id, m.name  from media_type m
inner join track t  on m.media_type_id = t.media_type_id),

track_protection as (select il.track_id , 
case 
when t.name in ('Protected AAC audio file','Protected MPEG-4 video file') then 'Yes'
else 'No'
end as 'protection'
from invoice_line il 
inner join track_media t on il.track_id =t.track_id )

select tp.protection, count(*) track_count,
round(cast(count(*) as float)*100/(select count(*) from track_protection),2) percent from track_protection tp
group by tp.protection


Done.


protection,track_count,percent
No,4315,90.71
Yes,442,9.29


## Compare Protected vs Non protected purchases

There are more purchases for non protected files comapred to protected. There is no value addition for the protected tracks.

In [219]:
%%sql
with invoice_tracks as
(select distinct(track_id) from invoice_line),

track_media as 
(select t.track_id, m.name,t.album_id  from media_type m
inner join track t  on m.media_type_id = t.media_type_id),

purchases as (select 
case 
 when (select it.track_id from invoice_tracks it where t.track_id =it.track_id  ) is  null then "No"
else 'Yes'    
end as 'purchased',
 case 
when t.name in ('Protected AAC audio file','Protected MPEG-4 video file') then 'Yes'
else 'No'
end as 'protection'             
              
from album a 
left  join track_media t on a.album_id = t.album_id )

select p.protection,p.purchased,count(*) track_count, 
round(cast(count(*) as float)*100/(select count(*) from purchases),2) percent

from purchases p 
group by p.purchased,p.protection
order by 1 desc,2 desc



Done.


protection,purchased,track_count,percent
Yes,Yes,154,4.4
Yes,No,297,8.48
No,Yes,1652,47.16
No,No,1400,39.97


The percent of Protcted purchase is low comapared to not protected 